In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [6]:
votes_2020 = pd.read_csv("2020_US_County_Level_Presidential_Results.csv")

In [10]:
votes_2020

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,Rep2020
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,1
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,1
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,1
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,1
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,1
...,...,...,...,...,...,...,...,...,...,...,...
3147,Wyoming,56037,Sweetwater County,12229,3823,16603,8406,0.736554,0.230260,0.506294,1
3148,Wyoming,56039,Teton County,4341,9848,14677,-5507,0.295769,0.670982,-0.375213,0
3149,Wyoming,56041,Uinta County,7496,1591,9402,5905,0.797277,0.169219,0.628058,1
3150,Wyoming,56043,Washakie County,3245,651,4012,2594,0.808824,0.162263,0.646560,1


In [9]:
def dummyvote(row):
    if row['votes_gop'] > row['votes_dem']:
        val = 1
    else:
        val = 0
    return val


votes_2020['Rep2020'] = votes_2020.apply(dummyvote, axis=1)

In [11]:
votes_2016 = pd.read_csv("2016result.csv")

In [13]:
def dummyvote(row):
    if row['votes_gop'] > row['votes_dem']:
        val = 1
    else:
        val = 0
    return val


votes_2016['Rep2016'] = votes_2016.apply(dummyvote, axis=1)

In [18]:
votes_2016

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,county_fips,Rep2016
0,0,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2013,1
1,1,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2016,1
2,2,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2020,1
3,3,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2050,1
4,4,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2060,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,3233.0,12153.0,16661.0,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,56037,1
3137,3137,7313.0,3920.0,12176.0,0.600608,0.321945,"3,393",27.87%,WY,Teton County,56039,0
3138,3138,1202.0,6154.0,8053.0,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,56041,1
3139,3139,532.0,2911.0,3715.0,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,56043,1


In [17]:
votes_2016 = votes_2016.rename(columns={'combined_fips': 'county_fips'})

In [20]:
df3 = votes_2020.merge(votes_2016, on='county_fips', how='inner', suffixes=('_1', '_2'))

In [21]:
df3

,state_name,county_fips,county_name_1,votes_gop_1,votes_dem_1,total_votes_1,diff_1,per_gop_1,per_dem_1,per_point_diff_1,...,votes_dem_2,votes_gop_2,total_votes_2,per_dem_2,per_gop_2,diff_2,per_point_diff_2,state_abbr,county_name_2,Rep2016
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,...,5908.0,18110.0,24661.0,0.239569,0.734358,"12,202",49.48%,AL,Autauga County,1
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,...,18409.0,72780.0,94090.0,0.195653,0.773515,"54,371",57.79%,AL,Baldwin County,1
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,...,4848.0,5431.0,10390.0,0.466603,0.522714,583,5.61%,AL,Barbour County,1
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,...,1874.0,6733.0,8748.0,0.214220,0.769662,"4,859",55.54%,AL,Bibb County,1
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,...,2150.0,22808.0,25384.0,0.084699,0.898519,"20,658",81.38%,AL,Blount County,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,Wyoming,56037,Sweetwater County,12229,3823,16603,8406,0.736554,0.230260,0.506294,...,3233.0,12153.0,16661.0,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,1
3107,Wyoming,56039,Teton County,4341,9848,14677,-5507,0.295769,0.670982,-0.375213,...,7313.0,3920.0,12176.0,0.600608,0.321945,"3,393",27.87%,WY,Teton County,0
3108,Wyoming,56041,Uinta County,7496,1591,9402,5905,0.797277,0.169219,0.628058,...,1202.0,6154.0,8053.0,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,1
3109,Wyoming,56043,Washakie County,3245,651,4012,2594,0.808824,0.162263,0.646560,...,532.0,2911.0,3715.0,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,1


In [23]:
election_result = df3[['county_fips','Rep2020',"Rep2016"]]

In [24]:
election_result

,county_fips,Rep2020,Rep2016
0,1001,1,1
1,1003,1,1
2,1005,1,1
3,1007,1,1
4,1009,1,1
...,...,...,...
3106,56037,1,1
3107,56039,0,0
3108,56041,1,1
3109,56043,1,1


In [26]:
election_result.to_csv("election_result.csv", index=False)